In [34]:
import re
import plotly 
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username='bakeralex664', api_key='a7vexR1Pli50bFWRXdJp')

# Analysis of Principle Component Analysis data generated by Cohen et al. 2018

While, determining the ranking the importance of the features feed to the Logistic Regression model an Principle Component Analysis (PCA) was conducted as an after thought. The aim of this PCA was to briefly observe potential clustering of Cancer and Normal data points and was projected into a 2D graph space (Fig S3 of Cohen et al. 2018). 

Here I am going to take this initial analysis further and explore potiential clusters of cancer stages and type projected in both a 2D and 3D graph space. The data used to generate these graph spaces will be generated either by PCA and T-SNE data compression methods.

- **PCA Overview:** PCA is a Linear dimensionality reduction method that uses Singular Value Decomposition of the data to project it to a lower dimensional space.
- **T-SNE Overview:** is a visualization tool of high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

# Loading and Cleaning Data for PCA and T-SNE

In [76]:
# Here i am merging all of the relavent data needed to conduct PCA and T-SNE into one dataframe
gene_data = pd.read_csv('../data/Cohen_S5.csv')
protein_data = pd.read_csv('../data/Cohen_S6.csv')

protein_data = protein_data.drop(['Unnamed: 0'], axis=1)

In [77]:
# data processing functions that maybe reused later on
def convert_to_numbers(rec):
    """
        Overview: This function is responsible for converting all of the columns that pandas dtypes object. Typically,
        these columns are measuring protein concentrations into a float numerical value. This function extracts numbers
        via regular expressions and the combines them and convert them into a float data type.
    """
    digit = 0
    if type(rec) is str:
        pre_decimal = ''
        digits = re.findall('\d+', rec)
        for indx, num in enumerate(digits):
            if (indx + 1) != len(digits):
                pre_decimal = pre_decimal + num
            else:
                digit = '.'.join([pre_decimal, num])
    return float(digit)

def conversion(df, skip_col, conv_col, func=convert_to_numbers):
    """
        Overview: This function is responsible for looping through select columns a pandas dataframe for further 
        processing.
    """
    
    for col in conv_col:
    
        if col in skip_col:
            continue

        df[col] = df[col].apply(func)

    return df

In [78]:
omega = gene_data['Ω score']
protein_data = protein_data
protein_data.tail()

,Patient ID #,Sample ID #,Tumor type,AJCC Stage,AFP (pg/ml),Angiopoietin-2 (pg/ml),AXL (pg/ml),CA-125 (U/ml),CA 15-3 (U/ml),CA19-9 (U/ml),...,sFas (pg/ml),SHBG (nM),sHER2/sEGFR2/sErbB2 (pg/ml),sPECAM-1 (pg/ml),TGFa (pg/ml),Thrombospondin-2 (pg/ml),TIMP-1 (pg/ml),TIMP-2 (pg/ml),CancerSEEK Logistic Regression Score,CancerSEEK Test Result
1812,PAPA 1353,PAPA 1353 PLS 1,Ovary,I,879.498,1484.70,2096.76,24.82,10.30,42.39,...,207.24,115.24,5390.31,8538.58,16.89,599.40,167799.61,50128.60,0.98,Positive
1813,PAPA 1354,PAPA 1354 PLS 1,Ovary,I,1337.330,1607.90,852.37,5.58,9.80,16.44,...,207.24,147.17,7951.03,12966.19,16.89,599.40,123443.76,54066.98,1.00,Positive
1814,PAPA 1355,PAPA 1355 PLS 1,Ovary,III,879.498,1592.84,1044.45,30.48,8.48,16.44,...,207.24,104.63,2396.36,1901.41,16.89,599.40,104070.89,39844.02,1.00,Positive
1815,PAPA 1356,PAPA 1356 PLS 1,Ovary,II,879.498,5267.95,1445.69,1469.45,23.74,62.26,...,207.24,73.55,3079.81,5312.90,16.89,6864.33,110579.24,42921.13,1.00,Positive
1816,PAPA 1357,PAPA 1357 PLS 1,Ovary,III,879.498,3546.43,1493.32,1428.31,836.85,37.90,...,207.24,72.22,3967.55,4045.18,16.89,12877.10,88464.04,47219.24,1.00,Positive


In [106]:
# Extracting and Merging all of the features involved that the Logistic Regression Model used.
extract_cols = ['Patient ID #', 'Sample ID #', 'Tumor type', 'AJCC Stage', 'CA-125 (U/ml)', 'CEA (pg/ml)', 'CA19-9 (U/ml)', 'Prolactin (pg/ml)', 'HGF (pg/ml)', 'OPN (pg/ml)', 'Myeloperoxidase (ng/ml)', 'TIMP-1 (pg/ml)']
log_reg_features = protein_data[extract_cols]
log_reg_features['omega'] = omega
log_reg_features.tail()

/Users/alexanderbaker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Patient ID #,Sample ID #,Tumor type,AJCC Stage,CA-125 (U/ml),CEA (pg/ml),CA19-9 (U/ml),Prolactin (pg/ml),HGF (pg/ml),OPN (pg/ml),Myeloperoxidase (ng/ml),TIMP-1 (pg/ml),omega
1812,PAPA 1353,PAPA 1353 PLS 1,Ovary,I,24.82,0.914,42.39,58266.97,284.34,38603.84,30.18,167799.61,0.98
1813,PAPA 1354,PAPA 1354 PLS 1,Ovary,I,5.58,1179.500,16.44,187828.79,374.03,29994.01,39.62,123443.76,3.92
1814,PAPA 1355,PAPA 1355 PLS 1,Ovary,III,30.48,443.010,16.44,241440.02,309.22,93601.15,11.93,104070.89,7.96
1815,PAPA 1356,PAPA 1356 PLS 1,Ovary,II,1469.45,443.010,62.26,140145.70,1153.70,145116.62,64.83,110579.24,0.81
1816,PAPA 1357,PAPA 1357 PLS 1,Ovary,III,1428.31,443.010,37.90,111737.24,224.69,103657.44,15.10,88464.04,1.43


In [80]:
# converting all protein columns into float data types.
# model_features is a list of all of the columns measuring protein levels
model_features = ['CA-125 (U/ml)', 'CEA (pg/ml)', 'CA19-9 (U/ml)', 'Prolactin (pg/ml)', 'HGF (pg/ml)', 'OPN (pg/ml)', 'Myeloperoxidase (ng/ml)', 'TIMP-1 (pg/ml)', 'omega']
features = log_reg_features[model_features].fillna(0)
features.head()

,CA-125 (U/ml),CEA (pg/ml),CA19-9 (U/ml),Prolactin (pg/ml),HGF (pg/ml),OPN (pg/ml),Myeloperoxidase (ng/ml),TIMP-1 (pg/ml),omega
0,5.090,540.1,16.452,11606.60000,377.26,56516.58,14.22,56428.71,2.96
1,7.270,5902.4,40.910,14374.99000,659.68,61001.39,23.88,73940.49,2.45
2,4.854,973.8,16.452,0.38375,329.07,88896.24,12.02,22797.28,1.22
3,5.390,2027.5,16.452,12072.51000,266.66,42549.61,6.49,20441.19,1.64
4,4.854,614.5,16.452,23718.17000,370.88,24274.11,13.33,56288.51,1.33


# Visualizing data used in Logistic Regression via PCA and T-SNE in 2D and 3D


In [107]:
# reusable functions related to generating figures both 2D and 3D

def gen_scatter(labels, data, y, x_label='Principal Component 1', 
               y_label='Principal Component 2', file='test_saving.png'):
    """
        Overview: gen_scatter is responsible for generating 2-Dimensional scatter plots. It does this by using 
        the matplotlib library.
        
        Inputs:
            - labels: a list/tuple of labels that are used to label the data in the generated figure
            - data: a numpy array representing the data that is being plotted
            - y: a numpy array that contains the labels of the x numpy array.
            - x_label: label for the x-axis
            - y_label: label for the y-axis
            - file: file path were saving the figure to.
    """
    with plt.style.context('seaborn-whitegrid'):
        plt.figure(figsize=(20,10))
        for lab in labels:
            plt.scatter(data[y==lab, 0],
                        data[y==lab, 1],
                        label=lab)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.legend()
        plt.tight_layout()
        plt.savefig(file)

def gen_3d_plotly(labels, data, y, skip_col='none', file='testing-3D.html'):
    """
        Overview: gen_3d_plotly is responsible for generating 3-Dimensional scatter plots. It does this by using 
        the plotly library.
        
        Inputs:
            - labels: a list/tuple of labels that are used to label the data in the generated figure
            - data: a numpy array representing the data that is being plotted
            - y: a numpy array that contains the labels of the x numpy array.
            - file: file path were saving the figure to.
    """
    
    traces = []
    for lab in labels:
        
        if lab in skip_col:
            continue
        
        trace = go.Scatter3d(
            x=data[y==lab, 0],
            y=data[y==lab, 1],
            z=data[y==lab, 2],
            name=lab,
            mode='markers')
        
        traces.append(trace)

    layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
    fig = go.Figure(data=traces, layout=layout)
    plot(fig, filename=file)
    iplot(fig, filename=file)
    

In [41]:
# For PCA and T-SNE to work properly we need to standardize the data. 
# Data standardization is required when you use different methods of measurement within a dataset.
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(features)
log_reg_features['AJCC Stage'] = log_reg_features['AJCC Stage'].fillna('normal')

# grabbing the y variables for labeling plot data.
y_types = log_reg_features['Tumor type'].values
y_stages = log_reg_features['AJCC Stage'].values

# labels will be using the figures generated
stages = ('I', 'II', 'III', 'normal')
types = ('Colorectum', 'Lung', 'Breast', 'Pancreas', 'Ovary', 'Esophagus', 'Liver', 'Stomach', 'Normal')

/Users/alexanderbaker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [104]:
# conducting a Principle Compenent Analysis (PCA)
pca_2d = PCA(n_components=2)
pca_3d = PCA(n_components=3)
pca_2d_data = pca_2d.fit_transform(features)
pca_3d_data = pca_3d.fit_transform(features)

In [105]:
# Generating figure for 2D and 3D figures from the PCA transformed data
gen_scatter(types, pca_2d_data, y_types, file='pca_2d_types.png')
gen_3d_plotly(types, pca_3d_data, y_types, file='pca_3d_types.html')
gen_scatter(stages, pca_2d_data, y_stages, file='pca_2d_stages.png')
gen_3d_plotly(stages, pca_3d_data, y_stages, file='pca_3d_stages.html')

/Users/alexanderbaker/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [102]:
# compressing data with t-sne 
tsne_calc = TSNE(n_components=2).fit_transform(features)
tsne_calc_3D = TSNE(n_components=3).fit_transform(features)

In [103]:
# Generating figure for 2D and 3D figures for T-SNE compressed data
gen_scatter(types, tsne_calc, y_types, file='tsne_2d_types.png')
gen_3d_plotly(types, tsne_calc_3D, y_types, file='tsne_3d_types.html')
gen_scatter(stages, tsne_calc, y_stages, file='tsne_2d_stages.png')
gen_3d_plotly(stages, tsne_calc_3D, y_stages, file='tsne_3d_stages.html')

# Conducting Analysis of Dimesion Reduction of RandomForest Features

In [108]:
forest_df = protein_data
forest_df['omega'] = omega
forest_df.head()

,Patient ID #,Sample ID #,Tumor type,AJCC Stage,AFP (pg/ml),Angiopoietin-2 (pg/ml),AXL (pg/ml),CA-125 (U/ml),CA 15-3 (U/ml),CA19-9 (U/ml),...,SHBG (nM),sHER2/sEGFR2/sErbB2 (pg/ml),sPECAM-1 (pg/ml),TGFa (pg/ml),Thrombospondin-2 (pg/ml),TIMP-1 (pg/ml),TIMP-2 (pg/ml),CancerSEEK Logistic Regression Score,CancerSEEK Test Result,omega
0,CRC 455,CRC 455 PLS 1,Colorectum,I,1583.450,5598.50,3621.04,5.090,19.08,16.452,...,55.06,6832.0700,9368.53,16.086,21863.74,56428.71,39498.82,0.938,Positive,2.96
1,CRC 456,CRC 456 PLS 1,Colorectum,I,715.308,20936.35,2772.96,7.270,10.04,40.910,...,72.92,5549.4700,6224.55,16.086,29669.66,73940.49,41277.09,0.925,Positive,2.45
2,CRC 457,CRC 457 PLS 1,Colorectum,II,4365.530,2350.93,4120.77,4.854,16.96,16.452,...,173.78,3698.1600,4046.48,179.030,6020.47,22797.28,28440.60,0.852,Negative,1.22
3,CRC 458,CRC 458 PLS 1,Colorectum,II,715.308,1604.34,2029.96,5.390,8.31,16.452,...,29.47,0.5856,6121.93,16.086,4331.02,20441.19,25896.73,0.618,Negative,1.64
4,CRC 459,CRC 459 PLS 1,Colorectum,II,801.300,2087.57,2069.17,4.854,11.73,16.452,...,78.07,5447.9300,6982.32,16.086,2311.91,56288.51,49425.20,0.318,Negative,1.33


In [109]:
# forest_df = conversion(forest_df, ['Patient ID #', 'Sample ID #', 'Tumor type', 'AJCC Stage', 'sPECAM-1 (pg/ml)', 'omega', 'TIMP-2 (pg/ml)', 'AXL (pg/ml)'],
#           forest_df.columns)
forest_features = forest_df.drop(['AJCC Stage', 'Tumor type', 'Sample ID #', 'Patient ID #', 'CancerSEEK Logistic Regression Score', 'CancerSEEK Test Result'],axis=1)
forest_features = forest_features.fillna(0)

In [110]:
forest_std = StandardScaler().fit_transform(forest_features)

In [115]:
# grabbing the y variables for labeling plot data. I am reusing
y_types = forest_df['Tumor type'].values
y_stages = forest_df['AJCC Stage'].values

# labels will be using the figures generated. I am reusing
stages = ('I', 'II', 'III', 'normal')
types = ('Colorectum', 'Lung', 'Breast', 'Pancreas', 'Ovary', 'Esophagus', 'Liver', 'Stomach', 'Normal')

In [116]:
forest_pca_2d = PCA(n_components=2).fit_transform(forest_std)
forest_pca_3d = PCA(n_components=3).fit_transform(forest_std)

In [117]:
gen_scatter(types, forest_pca_2d, y_types, file='forest_2d_pca_types.png')
gen_3d_plotly(types, forest_pca_3d, y_types, file='forest_3d_pca_types.html')
gen_scatter(stages, forest_pca_2d, y_stages, file='forest_2d_pca_stages.png')
gen_3d_plotly(stages, forest_pca_3d, y_stages, file='forest_3d_pca_stages.html')

/Users/alexanderbaker/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [118]:
forest_tsne_2d = TSNE(n_components=2).fit_transform(forest_std)
forest_tsne_3d = TSNE(n_components=3).fit_transform(forest_std)

In [119]:
gen_scatter(types, forest_tsne_2d, y_types, file='forest_2d_tsne_types.png')
gen_3d_plotly(types, forest_tsne_3d, y_types, file='forest_3d_tsne_types.html')
gen_scatter(stages, forest_tsne_2d, y_stages, file='forest_2d_tsne_stages.png')
gen_3d_plotly(stages, forest_tsne_3d, y_stages, file='forest_3d_tsne_stages.html')

/Users/alexanderbaker/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [101]:
forest_features.columns

Index(['AFP (pg/ml)', 'Angiopoietin-2 (pg/ml)', 'AXL (pg/ml)', 'CA-125 (U/ml)',
       'CA 15-3 (U/ml)', 'CA19-9 (U/ml)', 'CD44 (ng/ml)', 'CEA (pg/ml)',
       'CYFRA 21-1 (pg/ml)', 'DKK1 (ng/ml)', 'Endoglin (pg/ml)',
       'FGF2 (pg/ml)', 'Follistatin (pg/ml)', 'Galectin-3 (ng/ml)',
       'G-CSF (pg/ml)', 'GDF15 (ng/ml)', 'HE4 (pg/ml)', 'HGF (pg/ml)',
       'IL-6 (pg/ml)', 'IL-8 (pg/ml)', 'Kallikrein-6 (pg/ml)',
       'Leptin (pg/ml)', 'Mesothelin (ng/ml)', 'Midkine (pg/ml)',
       'Myeloperoxidase (ng/ml)', 'NSE (ng/ml)', 'OPG (ng/ml)', 'OPN (pg/ml)',
       'PAR (pg/ml)', 'Prolactin (pg/ml)', 'sEGFR (pg/ml)', 'sFas (pg/ml)',
       'SHBG (nM)', 'sHER2/sEGFR2/sErbB2 (pg/ml)', 'sPECAM-1 (pg/ml)',
       'TGFa (pg/ml)', 'Thrombospondin-2 (pg/ml)', 'TIMP-1 (pg/ml)',
       'TIMP-2 (pg/ml)', 'omega'],
      dtype='object')